Variable Coefficients
===
$\DeclareMathOperator{\opdiv}{div}$

A setup with varying heat-conductivity is modeled by the equation

$$
-\opdiv \lambda(x) \nabla u(x) = f(x),
$$

where $\lambda$ is the heat-conductivity, relating the heat-flux 

$$
q = -\lambda \nabla u
$$

to the temperature gradient $\nabla u$.

In the case of discontinuous $\lambda$, the equation is understood in distributional sense. This includes the interface conditions: The temperature on the left and right side are equal, and the heat flux leaving the left side enters the right side:

\begin{eqnarray*}
u_l & = & u_r \\
\lambda_l \frac{\partial u_r}{\partial n} & = & \lambda_r \frac{\partial u_r}{\partial n}
\end{eqnarray*}

The variational form is: find $u \in H^1(\Omega)$

$$
\int_\Omega \lambda(x) \nabla u \nabla v = \int_\Omega f v
$$

There is no issue with discontinuous coefficients. Both interface conditions are included: 
* continuity of temperature $u$ by the continuity of the trial space
* continuity of the heat flux in weak sense, such as Neumann boundary conditions

In [2]:
import netgen.gui
from ngsolve import *
%gui tk

Make a 2D geometry:

In [109]:
from netgen.geom2d import *
geo = SplineGeometry()
geo.AddRectangle( (0,0), (1,1), leftdomain=1, rightdomain=0, 
                 bcs=['b','r','t','l'])
geo.AddCircle( (0.3,0.7), 0.1, leftdomain=2, rightdomain=1)
geo.AddRectangle ( (0.2,0.2), (0.9,0.3), leftdomain=3, rightdomain=1)
geo.SetMaterial(1, "air")
geo.SetMaterial(2, "source")
geo.SetMaterial(3, "bar")

mesh = Mesh(geo.GenerateMesh(maxh=0.03))
mesh.Curve(3)
Draw (mesh)

In [110]:
print (mesh.GetMaterials())

('air', 'source', 'bar')


In [111]:
print (mesh.GetBoundaries())

('b', 'r', 't', 'l', 'default', 'default', 'default', 'default', 'default', 'default', 'default', 'default')


In [112]:
fes = H1(mesh, order=3, dirichlet="b|r")
u = fes.TrialFunction()
v = fes.TestFunction()

make a coefficient function taking one constant per material. Uses Python list comprehension.

In [113]:
lamvalues = { "air" : 1, "bar" : 100, "source" : 2 }
lam = CoefficientFunction( 
    [lamvalues[mat] for mat in mesh.GetMaterials()])
Draw (lam, mesh, "lambda")

In [114]:
lamvalues

{'air': 1, 'bar': 100, 'source': 2}

use conductivity in bilinear-form:

In [115]:
a = BilinearForm(fes)
a += SymbolicBFI (lam * grad(u) * grad(v))
f = LinearForm(fes)
f += SymbolicLFI (1 * v, definedon=mesh.Materials("source"))
a.Assemble()
f.Assemble()

In [116]:
reg = mesh.Materials("source|bar")
print (reg.VB(), reg.Mask())

VorB.VOL 0: 011


In [117]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw (gfu, mesh, "temperature")
Draw (grad(gfu), mesh, "gradient")
Draw (-lam*grad(gfu), mesh, "heatflux")